In [1]:
import pandas as pd
import pymongo
import numpy as np

In [2]:
%run ../scripts/mongo_connect.py
client = mongo_connect()
db = client.swatford

In [3]:
mrk_ref = pd.read_table("../data/MGI/MRK_Reference.rpt",header=None,names=["mgi_marker_acc_id",
                                                                           "marker_symbol",
                                                                           "marker_name",
                                                                           "marker_syns",
                                                                           "pmid"])

In [4]:
ref_data = {"acc":[],"pmid":[]}

In [5]:
for tup in mrk_ref[["mgi_marker_acc_id","pmid"]].itertuples():
    pmids = [int(x) for x in tup.pmid.split("|")]
    ref_data["acc"]+=[tup.mgi_marker_acc_id]*len(pmids)
    ref_data["pmid"]+=pmids
#     print([tup.mgi_marker_acc_id]*len(pmids),pmids)

In [6]:
mrk_gene = pd.read_table("../data/MGI/MGI_EntrezGene.rpt",header=None,names=["mgi_marker_acc_id",
                                                                             "marker_symbol",
                                                                             "status",
                                                                             "marker_name",
                                                                             "cm_pos",
                                                                             "chromosome",
                                                                             "type",
                                                                             "sec_acc",
                                                                             "gene_id",
                                                                             "syns",
                                                                             "feature_type",
                                                                             "genome_coord_start",
                                                                             "genome_coord_end",
                                                                             "strand",
                                                                             "biotype"])

C:\Users\SWatford\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (5,7,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
gene_pmid = mrk_gene[mrk_gene.gene_id.isnull()==False][["mgi_marker_acc_id",
                                            "gene_id"]].merge(pd.DataFrame(ref_data),
                                                              how="inner",
                                                              left_on="mgi_marker_acc_id",
                                                              right_on="acc"
                                                             )

In [8]:
to_insert = []
for i,row in gene_pmid[["gene_id","acc","pmid"]].iterrows():
    to_insert.append(dict(row))
    if i%25000==0:
        db.mgi.insert_many(to_insert)
        to_insert=[]
db.mgi.insert_many(to_insert)